In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6505,2020-12-29T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1093,169,1262,32147,...,2352,NaN,NaN,91261,1203327,802862.0,NaN,8.0,NaN,NaN
6506,2020-12-29T17:00:00,ITA,9,Toscana,43.769231,11.255889,879,157,1036,8979,...,3632,NaN,NaN,119236,1861004,1054622.0,NaN,6.0,Positivi diagnosticati solo con test antigenic...,NaN
6507,2020-12-29T17:00:00,ITA,10,Umbria,43.106758,12.388247,263,38,301,3267,...,613,NaN,NaN,28313,496881,250303.0,NaN,8.0,NaN,NaN
6508,2020-12-29T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,73,1,74,339,...,378,NaN,NaN,7230,62194,37216.0,NaN,0.0,Con riferimento alla nota inserita in data di ...,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6505,2020-12-29T17:00:00,19,Sicilia,1093,169,1262,32147,33409,163,995,55500,2352,NaN,NaN,91261,1203327,802862.0,8.0,NaN,NaN
6506,2020-12-29T17:00:00,9,Toscana,879,157,1036,8979,10015,-269,271,105589,3632,NaN,NaN,119236,1861004,1054622.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
6507,2020-12-29T17:00:00,10,Umbria,263,38,301,3267,3568,10,161,24132,613,NaN,NaN,28313,496881,250303.0,8.0,NaN,NaN
6508,2020-12-29T17:00:00,2,Valle d'Aosta,73,1,74,339,413,-81,0,6439,378,NaN,NaN,7230,62194,37216.0,0.0,Con riferimento alla nota inserita in data di ...,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-12-29', '2020-12-28')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-29T17:00:00,1,Piemonte,3029,196,3225,26158,29383,-920,840,158172,7860,NaN,NaN,195415,1664929,978418.0,7.0,NaN,NaN
1,2020-12-29T17:00:00,2,Valle d'Aosta,73,1,74,339,413,-81,0,6439,378,NaN,NaN,7230,62194,37216.0,0.0,Con riferimento alla nota inserita in data di ...,NaN
2,2020-12-29T17:00:00,3,Lombardia,3634,498,4132,50286,54418,-347,843,393995,24958,NaN,NaN,473371,4802831,2591103.0,45.0,NaN,NaN
3,2020-12-29T17:00:00,5,Veneto,2678,366,3044,87321,90365,-577,2655,149426,6298,NaN,NaN,246089,3258961,1203915.0,63.0,NaN,NaN
4,2020-12-29T17:00:00,6,Friuli Venezia Giulia,655,60,715,11062,11777,-335,493,34958,1612,NaN,NaN,48347,916889,337351.0,7.0,NaN,NaN
5,2020-12-29T17:00:00,7,Liguria,711,63,774,4985,5759,71,305,50956,2867,NaN,NaN,59582,701035,320126.0,7.0,I 151199 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-29T17:00:00,8,Emilia-Romagna,2598,221,2819,53993,56812,-1486,894,103543,7614,NaN,NaN,167969,2538045,1216598.0,13.0,NaN,NaN
7,2020-12-29T17:00:00,9,Toscana,879,157,1036,8979,10015,-269,271,105589,3632,NaN,NaN,119236,1861004,1054622.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-29T17:00:00,10,Umbria,263,38,301,3267,3568,10,161,24132,613,NaN,NaN,28313,496881,250303.0,8.0,NaN,NaN
9,2020-12-29T17:00:00,11,Marche,465,61,526,9603,10129,392,508,28661,1546,NaN,NaN,40336,519600,308324.0,7.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-28T17:00:00,1,Piemonte,3042,205,3247,27056,30303,-759,313,156468,7804,NaN,NaN,194575,1657292,976214.0,4.0,NaN,NaN
1,2020-12-28T17:00:00,2,Valle d'Aosta,95,4,99,395,494,62,61,6372,376,NaN,NaN,7242,61848,37107.0,2.0,Si segnala che a causa di un problema tecnico ...,NaN
2,2020-12-28T17:00:00,3,Lombardia,3791,512,4303,50462,54765,-4562,573,392854,24909,NaN,NaN,472528,4791224,2587261.0,13.0,NaN,NaN
3,2020-12-28T17:00:00,5,Veneto,2696,355,3051,87891,90942,921,2782,146385,6107,NaN,NaN,243434,3243320,1200115.0,16.0,NaN,NaN
4,2020-12-28T17:00:00,6,Friuli Venezia Giulia,645,56,701,11411,12112,-78,187,34151,1591,NaN,NaN,47854,909588,335797.0,9.0,NaN,NaN
5,2020-12-28T17:00:00,7,Liguria,718,61,779,4909,5688,-81,173,50737,2852,NaN,NaN,59277,696992,319115.0,0.0,I 148059 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-28T17:00:00,8,Emilia-Romagna,2642,220,2862,55436,58298,123,750,101255,7524,NaN,NaN,167077,2518742,1212763.0,15.0,NaN,NaN
7,2020-12-28T17:00:00,9,Toscana,890,164,1054,9230,10284,-516,181,105064,3617,NaN,NaN,118965,1853202,1052016.0,5.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-28T17:00:00,10,Umbria,268,34,302,3256,3558,5,88,23984,610,NaN,NaN,28152,493224,249326.0,1.0,NaN,NaN
9,2020-12-28T17:00:00,11,Marche,454,60,514,9223,9737,66,131,28556,1535,NaN,NaN,39828,515371,305671.0,3.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-29T17:00:00,1,Piemonte,3029,196,3225,26158,29383,-920,840,158172,7860,NaN,NaN,195415,1664929,978418.0,7.0,NaN,NaN
1,2020-12-29T17:00:00,2,Valle d'Aosta,73,1,74,339,413,-81,0,6439,378,NaN,NaN,7230,62194,37216.0,0.0,Con riferimento alla nota inserita in data di ...,NaN
2,2020-12-29T17:00:00,3,Lombardia,3634,498,4132,50286,54418,-347,843,393995,24958,NaN,NaN,473371,4802831,2591103.0,45.0,NaN,NaN
3,2020-12-29T17:00:00,4,Trentino-Alto Adige,516,68,584,11854,12438,10,266,36244,1655,0.0,0.0,50337,802451,312123.0,7.0,0,NaN
4,2020-12-29T17:00:00,5,Veneto,2678,366,3044,87321,90365,-577,2655,149426,6298,NaN,NaN,246089,3258961,1203915.0,63.0,NaN,NaN
5,2020-12-29T17:00:00,6,Friuli Venezia Giulia,655,60,715,11062,11777,-335,493,34958,1612,NaN,NaN,48347,916889,337351.0,7.0,NaN,NaN
6,2020-12-29T17:00:00,7,Liguria,711,63,774,4985,5759,71,305,50956,2867,NaN,NaN,59582,701035,320126.0,7.0,I 151199 test antigenici rapidi sono riferiti ...,NaN
7,2020-12-29T17:00:00,8,Emilia-Romagna,2598,221,2819,53993,56812,-1486,894,103543,7614,NaN,NaN,167969,2538045,1216598.0,13.0,NaN,NaN
8,2020-12-29T17:00:00,9,Toscana,879,157,1036,8979,10015,-269,271,105589,3632,NaN,NaN,119236,1861004,1054622.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2020-12-29T17:00:00,10,Umbria,263,38,301,3267,3568,10,161,24132,613,NaN,NaN,28313,496881,250303.0,8.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2020-12-28T17:00:00,1,Piemonte,3042,205,3247,27056,30303,-759,...,156468,7804,NaN,NaN,194575,1657292,976214.0,4.0,NaN,NaN
1,1,2020-12-28T17:00:00,2,Valle d'Aosta,95,4,99,395,494,62,...,6372,376,NaN,NaN,7242,61848,37107.0,2.0,Si segnala che a causa di un problema tecnico ...,NaN
2,2,2020-12-28T17:00:00,3,Lombardia,3791,512,4303,50462,54765,-4562,...,392854,24909,NaN,NaN,472528,4791224,2587261.0,13.0,NaN,NaN
3,19,2020-12-28T17:00:00,4,Trentino-Alto Adige,527,65,592,11836,12428,73,...,36007,1636,0.0,0.0,50071,799599,311469.0,3.0,0,NaN
4,3,2020-12-28T17:00:00,5,Veneto,2696,355,3051,87891,90942,921,...,146385,6107,NaN,NaN,243434,3243320,1200115.0,16.0,NaN,NaN
5,4,2020-12-28T17:00:00,6,Friuli Venezia Giulia,645,56,701,11411,12112,-78,...,34151,1591,NaN,NaN,47854,909588,335797.0,9.0,NaN,NaN
6,5,2020-12-28T17:00:00,7,Liguria,718,61,779,4909,5688,-81,...,50737,2852,NaN,NaN,59277,696992,319115.0,0.0,I 148059 test antigenici rapidi sono riferiti ...,NaN
7,6,2020-12-28T17:00:00,8,Emilia-Romagna,2642,220,2862,55436,58298,123,...,101255,7524,NaN,NaN,167077,2518742,1212763.0,15.0,NaN,NaN
8,7,2020-12-28T17:00:00,9,Toscana,890,164,1054,9230,10284,-516,...,105064,3617,NaN,NaN,118965,1853202,1052016.0,5.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2020-12-28T17:00:00,10,Umbria,268,34,302,3256,3558,5,...,23984,610,NaN,NaN,28152,493224,249326.0,1.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2020-12-29T17:00:00,1,Piemonte,3029,196,3225,26158,29383,-920,840,...,7860,NaN,NaN,195415,1664929,978418.0,7.0,NaN,NaN,4341375
1,2020-12-29T17:00:00,2,Valle d'Aosta,73,1,74,339,413,-81,0,...,378,NaN,NaN,7230,62194,37216.0,0.0,Con riferimento alla nota inserita in data di ...,NaN,125501
2,2020-12-29T17:00:00,3,Lombardia,3634,498,4132,50286,54418,-347,843,...,24958,NaN,NaN,473371,4802831,2591103.0,45.0,NaN,NaN,10103969
3,2020-12-29T17:00:00,4,Trentino-Alto Adige,516,68,584,11854,12438,10,266,...,1655,0.0,0.0,50337,802451,312123.0,7.0,0,NaN,1074819
4,2020-12-29T17:00:00,5,Veneto,2678,366,3044,87321,90365,-577,2655,...,6298,NaN,NaN,246089,3258961,1203915.0,63.0,NaN,NaN,4907704
5,2020-12-29T17:00:00,6,Friuli Venezia Giulia,655,60,715,11062,11777,-335,493,...,1612,NaN,NaN,48347,916889,337351.0,7.0,NaN,NaN,1211357
6,2020-12-29T17:00:00,7,Liguria,711,63,774,4985,5759,71,305,...,2867,NaN,NaN,59582,701035,320126.0,7.0,I 151199 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2020-12-29T17:00:00,8,Emilia-Romagna,2598,221,2819,53993,56812,-1486,894,...,7614,NaN,NaN,167969,2538045,1216598.0,13.0,NaN,NaN,4467118
8,2020-12-29T17:00:00,9,Toscana,879,157,1036,8979,10015,-269,271,...,3632,NaN,NaN,119236,1861004,1054622.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2020-12-29T17:00:00,10,Umbria,263,38,301,3267,3568,10,161,...,613,NaN,NaN,28313,496881,250303.0,8.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2020-12-29T17:00:00,1,Piemonte,3029,196,3225,26158,29383,-920,840,...,NaN,NaN,195415,1664929,978418.0,7.0,NaN,NaN,4341375,575
1,2020-12-29T17:00:00,2,Valle d'Aosta,73,1,74,339,413,-81,0,...,NaN,NaN,7230,62194,37216.0,0.0,Con riferimento alla nota inserita in data di ...,NaN,125501,20
2,2020-12-29T17:00:00,3,Lombardia,3634,498,4132,50286,54418,-347,843,...,NaN,NaN,473371,4802831,2591103.0,45.0,NaN,NaN,10103969,1036
3,2020-12-29T17:00:00,4,Trentino-Alto Adige,516,68,584,11854,12438,10,266,...,0.0,0.0,50337,802451,312123.0,7.0,0,NaN,1074819,106
4,2020-12-29T17:00:00,5,Veneto,2678,366,3044,87321,90365,-577,2655,...,NaN,NaN,246089,3258961,1203915.0,63.0,NaN,NaN,4907704,1016
5,2020-12-29T17:00:00,6,Friuli Venezia Giulia,655,60,715,11062,11777,-335,493,...,NaN,NaN,48347,916889,337351.0,7.0,NaN,NaN,1211357,180
6,2020-12-29T17:00:00,7,Liguria,711,63,774,4985,5759,71,305,...,NaN,NaN,59582,701035,320126.0,7.0,I 151199 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2020-12-29T17:00:00,8,Emilia-Romagna,2598,221,2819,53993,56812,-1486,894,...,NaN,NaN,167969,2538045,1216598.0,13.0,NaN,NaN,4467118,634
8,2020-12-29T17:00:00,9,Toscana,879,157,1036,8979,10015,-269,271,...,NaN,NaN,119236,1861004,1054622.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2020-12-29T17:00:00,10,Umbria,263,38,301,3267,3568,10,161,...,NaN,NaN,28313,496881,250303.0,8.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-12-29T17:00:00,5,Veneto,2678,366,3044,87321,90365,-577,2655,...,36.02,-570,-127,15641,3800.0,191,3041,16.97,0.05410,5.01
1,2020-12-29T17:00:00,12,Lazio,2854,305,3159,71526,74685,-1385,1218,...,36.01,-1417,252,11196,5644.0,54,2549,10.88,0.02077,2.73
2,2020-12-29T17:00:00,19,Sicilia,1093,169,1262,32147,33409,163,995,...,28.74,140,345,8807,5164.0,26,806,11.30,0.02003,1.84
3,2020-12-29T17:00:00,8,Emilia-Romagna,2598,221,2819,53993,56812,-1486,894,...,34.86,-1443,144,19303,3835.0,90,2288,4.63,0.02001,3.76
4,2020-12-29T17:00:00,3,Lombardia,3634,498,4132,50286,54418,-347,843,...,48.07,-176,270,11607,3842.0,49,1141,7.26,0.00834,4.69
5,2020-12-29T17:00:00,1,Piemonte,3029,196,3225,26158,29383,-920,840,...,34.09,-898,527,7637,2204.0,56,1704,11.00,0.01935,4.50
6,2020-12-29T17:00:00,16,Puglia,1466,136,1602,51231,52833,-324,749,...,36.86,-293,104,9668,3637.0,34,1039,7.75,0.01869,2.19
7,2020-12-29T17:00:00,15,Campania,1402,97,1499,76992,78491,-805,625,...,19.21,-784,192,6849,2413.0,46,1384,9.13,0.01080,3.24
8,2020-12-29T17:00:00,11,Marche,465,61,526,9603,10129,392,508,...,42.66,380,377,4229,2653.0,11,105,12.01,0.03346,2.66
9,2020-12-29T17:00:00,6,Friuli Venezia Giulia,655,60,715,11062,11777,-335,493,...,33.33,-349,306,7301,1554.0,21,807,6.75,0.04070,3.99


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,68,38,106,64.15
1,Lombardia,498,538,1036,48.07
2,Marche,61,82,143,42.66
3,Puglia,136,233,369,36.86
4,Veneto,366,650,1016,36.02
5,Lazio,305,542,847,36.01
6,Emilia-Romagna,221,413,634,34.86
7,Piemonte,196,379,575,34.09
8,Friuli Venezia Giulia,60,120,180,33.33
9,Umbria,38,79,117,32.48


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,2020-12-27T17:00:00,22,P.A. Trento,356,45,401,1374,1775,49,86,18372,902,NaN,NaN,21049,442600,150012.0,1.0,NaN,"Dei 86 casi odierni, 26 sono molecolari con pr..."
616,2020-12-28T17:00:00,21,P.A. Bolzano,153,20,173,10434,10607,27,120,17594,722,NaN,NaN,28923,356239,161307.0,1.0,NaN,NaN
617,2020-12-28T17:00:00,22,P.A. Trento,374,45,419,1402,1821,46,99,18413,914,NaN,NaN,21148,443360,150162.0,2.0,NaN,"Dei 99 casi odierni, 25 sono molecolari con pr..."
618,2020-12-29T17:00:00,21,P.A. Bolzano,136,22,158,10520,10678,71,91,17607,729,NaN,NaN,29014,357337,161485.0,3.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,2020-12-25T17:00:00,4,Trentino-Alto Adige,453,68,521,11820,12341,1488,336,...,12.0,5001.0,1748.0,1516.0,13.05,4.22,6.72,19.22,3.03,13.71
306,2020-12-26T17:00:00,4,Trentino-Alto Adige,486,65,551,11741,12292,-49,148,...,12.0,1723.0,-500.0,-79.0,11.80,4.48,8.59,-29.60,-4.41,-0.40
307,2020-12-27T17:00:00,4,Trentino-Alto Adige,510,65,575,11780,12355,63,143,...,7.0,1376.0,337.0,39.0,11.30,4.65,10.39,42.43,0.00,0.51
308,2020-12-28T17:00:00,4,Trentino-Alto Adige,527,65,592,11836,12428,73,219,...,14.0,1850.0,552.0,56.0,10.98,4.76,11.84,39.67,0.00,0.59


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
